In [1]:
import psycopg2
import pandas as pd
import numpy as np

conn_string = ""
conn = psycopg2.connect(conn_string)
cur=conn.cursor()

cur.execute("SELECT * FROM capstone.cap_data;")
result = cur.fetchall()

In [2]:
columns = ["row_id","subject_id","hadm_id","intime","outtime","information",'readmission']
df = pd.DataFrame(result,columns = columns)
df.head(10)

,row_id,subject_id,hadm_id,intime,outtime,information,readmission
0,31792,25996,135999,2200-04-14 19:12:00,2200-04-16 18:11:00,[**2200-4-16**] 12:21 PM\n CT HEAD W/O CONTRAS...,0
1,55392,88857,136000,2109-12-05 21:48:10,2109-12-06 16:16:49,Chief Complaint: ETOH withdrawal\n I saw and...,0
2,5401,4461,136001,2127-02-02 20:52:14,2127-03-02 15:16:49,[**2127-2-4**] 5:38 AM\n CHEST (PORTABLE AP) ...,0
3,13893,11342,136006,2176-06-04 19:13:08,2176-06-06 22:21:21,NURSING MICU NOTE 7P-7A\n\n46Y/O FEMALE PRESEN...,0
4,7825,6428,136010,2116-05-27 10:47:22,2116-05-29 15:35:55,[**2116-5-29**] 1:42 PM\n [**Last Name (un) 26...,0
5,13091,10704,136012,2144-12-21 16:32:06,2145-01-14 00:43:52,[**2144-12-24**] 5:30 AM\n CHEST (PORTABLE AP)...,0
6,46325,60106,136013,2145-01-25 21:04:18,2145-01-29 17:15:54,"Briefly, Ms. [**Known lastname 4702**] is a 55...",0
7,46325,60106,136013,2145-02-02 07:00:13,2145-02-04 22:19:03,"Briefly, Ms. [**Known lastname 4702**] is a 55...",0
8,49057,68807,136015,2146-09-23 06:58:03,2146-09-24 15:43:04,[**2146-9-24**] 5:26 AM\n CHEST (PORTABLE AP) ...,0
9,58775,99366,136021,2197-03-20 23:57:57,2197-03-28 17:48:50,[**2197-4-13**] 10:24 AM\n CT HEAD W/O CONTRAS...,0


In [4]:
import re

def preprocess(x):
    y=re.sub('\\[(.*?)\\]','',x) #remove de-identified brackets
    y=re.sub('[0-9]+\.','',y) #remove 1.2. since the segmenter segments based on this
    y=re.sub('dr\.','doctor',y)
    y=re.sub('m\.d\.','md',y)
    y=re.sub('admission date:','',y)
    y=re.sub('discharge date:','',y)
    y=re.sub('--|__|==','',y)
    # remove, digits, spaces
    result = " ".join(y.split())
    
    return result

In [5]:
total_lst = []
for info in df.iloc[:45942]['information']:
    info = info.split('\n')
    part_lst = []
    for sen in info:
        tmp=preprocess(sen)
        part_lst.append(tmp)
    total_lst.append(part_lst)
    
print(total_lst)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
total = np.array(total_lst)
data = pd.DataFrame(total,columns=["Text"])
print(f'df\'s shape:{df.shape}, data\'s shape:{data.shape}')
new_df = pd.concat([df,data],axis=1)
print(f'new_df\'s shape: {new_df.shape}')
new_df.head()

df's shape:(45942, 7), data's shape:(45942, 1)
new_df's shape: (45942, 8)


C:\Users\170890\Anaconda3\envs\nlp2\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


,row_id,subject_id,hadm_id,intime,outtime,information,readmission,Text
0,31792,25996,135999,2200-04-14 19:12:00,2200-04-16 18:11:00,[**2200-4-16**] 12:21 PM\n CT HEAD W/O CONTRAS...,0,"[12:21 PM, CT HEAD W/O CONTRAST Clip #, Reason..."
1,55392,88857,136000,2109-12-05 21:48:10,2109-12-06 16:16:49,Chief Complaint: ETOH withdrawal\n I saw and...,0,"[Chief Complaint: ETOH withdrawal, I saw and e..."
2,5401,4461,136001,2127-02-02 20:52:14,2127-03-02 15:16:49,[**2127-2-4**] 5:38 AM\n CHEST (PORTABLE AP) ...,0,"[5:38 AM, CHEST (PORTABLE AP) Clip #, Reason: ..."
3,13893,11342,136006,2176-06-04 19:13:08,2176-06-06 22:21:21,NURSING MICU NOTE 7P-7A\n\n46Y/O FEMALE PRESEN...,0,"[NURSING MICU NOTE 7P-7A, , 46Y/O FEMALE PRESE..."
4,7825,6428,136010,2116-05-27 10:47:22,2116-05-29 15:35:55,[**2116-5-29**] 1:42 PM\n [**Last Name (un) 26...,0,"[1:42 PM, DUP EXTEXT BIL (MAP/DVT) Clip #, Rea..."


In [9]:
from transformers import DataCollatorWithPadding, AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

tokenizer = AutoTokenizer.from_pretrained("bvanaken/CORe-clinical-mortality-prediction")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained("bvanaken/CORe-clinical-mortality-prediction", num_labels=2)

In [10]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

In [17]:
df_c = new_df.copy()
cols = ['row_id','subject_id','hadm_id','intime','outtime','information']
df_c.drop(cols,axis=1,inplace=True)
df_c.head()
train = new_df.iloc[:36754]
evalu = new_df.iloc[36754:]

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=evalu,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [19]:
trainer.train()

***** Running training *****
  Num examples = 36754
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 11490
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ········


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\170890/.netrc


KeyError: 29754